In [1]:
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, precision_recall_curve, average_precision_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline


In [2]:
# Tải data đã lưu
split_file = "splitted_data.pkl"
print("Tải dữ liệu đã lưu từ:", split_file)

with open(split_file, "rb") as f:
    data_splits = pickle.load(f)

# Lấy các tập dữ liệu từ dictionary
X_train = data_splits["X_train"]
X_val = data_splits["X_val"]
X_test = data_splits["X_test"]
y_train = data_splits["y_train"]
y_val = data_splits["y_val"]
y_test = data_splits["y_test"]
X_train_val = data_splits["X_train_val"]
y_train_val = data_splits["y_train_val"]

Tải dữ liệu đã lưu từ: splitted_data.pkl


In [ ]:

# Tạo pipeline với SMOTE và XGBoost
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('classifier', xgb.XGBClassifier(objective='binary:logistic', random_state=42))
])

# Định nghĩa không gian tìm kiếm siêu tham số
param_grid = {
    # Tham số cho SMOTE
    'smote__k_neighbors': [5, 10, 15],  # Số lượng neighbors để tạo mẫu tổng hợp
    'smote__sampling_strategy': [0.5, 0.75, 1.0],  # Tỷ lệ resampling (1.0 = cân bằng hoàn toàn)
    
    # Tham số cho XGBoost
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__max_depth': [3, 5, 7],
    'classifier__min_child_weight': [1, 3, 5],
    'classifier__gamma': [0, 0.1, 0.2],
    'classifier__subsample': [0.8, 0.9, 1.0],
    'classifier__colsample_bytree': [0.8, 0.9, 1.0],
    'classifier__scale_pos_weight': [1, 5, 10]  # Quan trọng cho dữ liệu mất cân bằng
}

# Đối với fraud detection, precision, recall, f1 và ROC-AUC đều quan trọng
# Nhưng average_precision (PR-AUC) thường phù hợp nhất cho dữ liệu mất cân bằng nặng
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='average_precision',  # Sử dụng PR-AUC
    cv=5,  # Stratified K-Fold mặc định sẽ được sử dụng
    verbose=2,
    n_jobs=-1
)

# Huấn luyện trên tập train
grid_search.fit(X_train, y_train)

# In ra siêu tham số tốt nhất
print("Siêu tham số tốt nhất:", grid_search.best_params_)
print("Điểm số tốt nhất:", grid_search.best_score_)

# Lấy mô hình tốt nhất
best_model = grid_search.best_estimator_

# Đánh giá trên tập test
y_test_pred = best_model.predict(X_test)

# In kết quả đánh giá trên từng tập
print("\nKết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))

# Lưu mô hình tốt nhất (chỉ lưu phần XGBoost)
best_xgb_model = best_model.named_steps['classifier']
best_xgb_model.save_model('best_fraud_detection_model.json')

Fitting 5 folds for each of 59049 candidates, totalling 295245 fits


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Bước 3: Áp dụng SMOTE trên tập train_val
# Sử dụng tham số SMOTE tốt nhất từ grid search
smote = SMOTE(
    k_neighbors=10,  # Thay thế bằng tham số tốt nhất từ grid search
    sampling_strategy=0.75,  # Thay thế bằng tham số tốt nhất
    random_state=42
)
X_train_val_resampled, y_train_val_resampled = smote.fit_resample(X_train_val, y_train_val)

# Bước 4: Huấn luyện lại mô hình trên dữ liệu train_val đã được resample
best_xgb_model.fit(
    X_train_val_resampled,
    y_train_val_resampled,
    eval_set=[(X_test, y_test)],
    eval_metric=['auc', 'logloss'],
    early_stopping_rounds=50,
    verbose=True
)

# Bước 5: Đánh giá mô hình trên tập test
y_test_pred = best_xgb_model.predict(X_test)

In [ ]:
print("\nBáo cáo phân loại chi tiết:")
print(classification_report(y_test, y_test_pred))
# Bước 6: Vẽ confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()
